In [1]:
import pandas  as pd
import numpy as np

In [2]:
producers=pd.read_csv('/Users/macbook/Desktop/BA6/Database/db2018imdb/producers.csv', sep=',')
people_old=pd.read_csv('/Users/macbook/Desktop/BA6/Database/newfiles/people.csv', sep=',')

In [3]:
cols_to_check = ['ClipIds','Roles','AddInfos']
producers[cols_to_check] = producers[cols_to_check].replace({'\[':'','\]':''},regex=True)

producers = producers.replace('', np.nan, regex=True)

producers.drop_duplicates(keep='first', inplace=True)
producers.drop_duplicates(subset=['FullName'], keep='first', inplace=True)

In [4]:
result = pd.merge(producers, people_old, how='left', on='FullName')
result = result[np.isnan(result['pid'])]
result['pid']=(result.FullName).rank(method='dense')+people_old['pid'].max()
result[['pid']] = result[['pid']].astype(int)
result= result[['pid','FullName']]
people = pd.concat([people_old, result])

In [5]:
people.to_csv('people.csv', sep=',', encoding='utf-8', index=False)

In [6]:
produce = pd.merge(producers, people, how='left', on='FullName')

In [7]:
produce = produce.drop(['FullName'],axis=1)

produce[['ClipIds']] = produce[['ClipIds']].astype(str)
produce[['AddInfos']] = produce[['AddInfos']].astype(str)
produce[['Roles']] = produce[['Roles']].astype(str)

def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = []
        i=0
        for c in target_column:
            split_row.append(row[c].split(separator))
            i+=1
            
        for s in range (0,len(split_row[0])):
            j=0
            new_row = row.to_dict()
            for c in target_column:
                new_row[c] = split_row[j][s]
                j+=1
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    global new_df
    new_df = pd.DataFrame(new_rows)
    return new_df

produce = splitDataFrameList( produce, ['ClipIds','AddInfos','Roles'] , '|')

In [8]:
produce = produce.replace('', np.nan, regex=True)
produce = produce.replace('nan', np.nan, regex=True)

produce = produce[['pid', 'ClipIds', 'Roles', 'AddInfos']]

produce.drop_duplicates(subset=['pid','ClipIds'], keep=False)

,pid,ClipIds,Roles,AddInfos
0,2006887,742507,producer,NaN
1,2006887,960777,producer,NaN
2,54,4016759,segment producer,2000
3,2148879,1431901,producer,NaN
4,67,714863,producer,NaN
5,2148880,214323,producer,NaN
6,2148881,36630,associate producer,NaN
7,2148882,576822,co-producer,NaN
8,85,1799926,producer,NaN
9,85,1833390,producer,NaN


In [9]:
produce.to_csv('produce.csv', sep=',', encoding='utf-8', index=False)